In [1]:
using LinearAlgebra
using Random
using Distributions
using Plots

In [16]:
#A = [3 4 0 0;4 65 8 0; 0 8 84 7; 0 0 7 3]
n = 10
d = rand(Uniform(-20,20),10)
sd = rand(Uniform(-20,20),10)
A = SymTridiagonal(d,sd)
A = convert(Array,A)

10×10 Matrix{Float64}:
 -7.21619   16.2968     0.0       …   0.0       0.0       0.0
 16.2968     2.85056  -17.5671        0.0       0.0       0.0
  0.0      -17.5671   -12.7417        0.0       0.0       0.0
  0.0        0.0       -0.990639      0.0       0.0       0.0
  0.0        0.0        0.0           0.0       0.0       0.0
  0.0        0.0        0.0       …   0.0       0.0       0.0
  0.0        0.0        0.0          19.6992    0.0       0.0
  0.0        0.0        0.0          -4.18474  -4.30629   0.0
  0.0        0.0        0.0          -4.30629   2.16778   7.50141
  0.0        0.0        0.0           0.0       7.50141  14.8183

In [17]:
eigvals(A)

10-element Vector{Float64}:
 -32.81391902974915
 -28.77771807845872
 -13.75639518680804
 -11.173595411114936
  -9.359322475843529
  -0.9800503754880567
   7.991310021061761
  18.33830706978845
  21.260935907019753
  27.866902899849617

In [4]:
function givensTri(A)
    line,column = size(A)
    output = deepcopy(A)
    Q = I
    for i = 2:line
        γ = output[i-1,i-1]/(sqrt(output[i,i-1]^2+output[i-1,i-1]^2))
        σ = output[i,i-1]/(sqrt(output[i,i-1]^2+output[i-1,i-1]^2))
        G = Matrix(1.0*I,size(A))
        G[i-1,i-1] = γ
        G[i-1,i] = σ
        G[i,i-1] = -σ
        G[i,i] = γ
        Q *= G'
        output = G * output
        output = zeroTresh(output)
    end
    return(Q,output)
end


givensTri (generic function with 1 method)

In [5]:
function zeroTresh(A)
    line,column = size(A)
    output = deepcopy(A)
    tol = 1e-12
    for i = 1: line
        for j = 1:column
            #A[i,j] = round(A[i,j], digits=3)
            if(-tol < A[i,j] < tol)
                output[i,j] = 0
            end
        end
    end
    return(output)
end

zeroTresh (generic function with 1 method)

In [6]:
function qrHouseholder(X)
    Q = I
    Qt = I
    u = X[:,1]-norm(X[:,1])*basisVector(size(X,1),1)
    v = u/norm(u)
    Qi = I-2*v*v'
    Qt = Qi*Qt
    Q*=Qi'
    for i = 2 : size(X,1)-1
        Qi = 1.0*Matrix(I,size(X))
        s = size(X,1)
        u = (Qt*X)[i:s,i] - norm((Qt*X)[i:s,i])*basisVector(size(X,1)-i+1,1)
        v = u/norm(u)
        Qip = I - 2*v*v'
        Qi[i:s,i:s] = Qip
        Qt = Qi*Qt
        Q *= Qi'
    end
    
    return Q,Qt*X
end
    

qrHouseholder (generic function with 1 method)

In [21]:
function checkConv(C)
    tol = 1e-5
    converge = false
    convergence = tr(C) - size(C,1)
    if( -tol <= convergence <= tol)
        converge = true
    end
    return converge
end
function qrConvTri(h)
    maxiter = 10000
    Q = I
    q1,r1 = givensTri(h)
    A1 = r1*q1
    Q *= q1
    q2,r2 = givensTri(A1)
    A2 = zeroTresh(r2*q2)
    C = A2/A1
    Q *= q2
    i = 0
    while(!checkConv(C) && i < maxiter)
        i += 1
        A1 = deepcopy(A2)
        q2,r2 = givensTri(A2)
        Q *= q2
        A2 = zeroTresh(r2*q2)
        C = A2/A1
    end
    r2 = q2*r2
    println("qrConvTri converged after ",i," iterations")
    return Q, r2
end
function qrConv(h)
    maxiter = 10000
    Q = I
    q1,r1 = qrHouseholder(h)
    A1 = r1*q1
    Q *= q1
    q2,r2 = qrHouseholder(A1)
    A2 = zeroTresh(r2*q2)
    C = A2/A1
    Q *= q2
    i = 0
    while(!checkConv(C) && i<maxiter)
        i += 1
        A1 = deepcopy(A2)
        q2,r2 = qrHouseholder(A2)
        Q *= q2
        A2 = zeroTresh(r2*q2)
        C = A2/A1
    end
    if(i == maxiter)
        @warn "Convergence hasnt been reached"
    else
        println("qrConv converged after ",i," iterations")
    end
    r2 = q2*r2
    return Q,r2
end

qrConv (generic function with 1 method)

In [8]:
function basisVector(n,i)
    e = zeros(n)
    e[i] = 1
    return e
end

basisVector (generic function with 1 method)

In [9]:
T = 1*Matrix(I,4,4)
T[2:3,2:3] = [4 5; 6 7]
T

4×4 Matrix{Int64}:
 1  0  0  0
 0  4  5  0
 0  6  7  0
 0  0  0  1

In [18]:
println(@elapsed Qtri,Rtri = qrConvTri(A))
println(@elapsed Q,R = qrConv(A))

qrConvTri converged after 53 iterations
0.0046221
qrConv converged after 53 iterations
0.0060533


In [19]:
eigvals(A)

10-element Vector{Float64}:
 -32.81391902974915
 -28.77771807845872
 -13.75639518680804
 -11.173595411114936
  -9.359322475843529
  -0.9800503754880567
   7.991310021061761
  18.33830706978845
  21.260935907019753
  27.866902899849617

In [20]:
zeroTresh(Rtri)

10×10 Matrix{Float64}:
 -28.7933      0.250581    0.0        …   0.0           0.0          0.0
   0.250581  -32.7983     -0.0200957      0.0           0.0          0.0
   0.0        -0.0200957  27.8549         0.0           0.0          0.0
   0.0         0.0        -0.281463       0.0           0.0          0.0
   0.0         0.0         0.0            0.0           0.0          0.0
   0.0         0.0         0.0        …   0.0           0.0          0.0
   0.0         0.0         0.0            0.000524226   0.0          0.0
   0.0         0.0         0.0           -9.35932      -3.19754e-6   0.0
   0.0         0.0         0.0           -3.19754e-6    7.99131      0.0
   0.0         0.0         0.0            0.0           0.0         -0.98005

In [26]:
X = range(3,stop = 70)
Tri = zeros(length(X))
Hou = zeros(length(X))
Mats = Matrix{Float64}[]
for i in X
    println("iter = ",i-2)
    d = rand(Uniform(-20,20),i)
    sd = rand(Uniform(-20,20),i)
    A = SymTridiagonal(d,sd)
    A = convert(Array,A)
    push!(Mats,A)
    Tri[i-2] = @elapsed qrConvTri(A)
    Hou[i-2] = @elapsed qrConv(A)
end


iter = 1
qrConvTri converged after 15 iterations
qrConv converged after 15 iterations
iter = 2
qrConvTri converged after 11 iterations
qrConv converged after 14 iterations
iter = 3
qrConvTri converged after 58 iterations
qrConv converged after 58 iterations
iter = 4
qrConvTri converged after 154 iterations
qrConv converged after 54 iterations
iter = 5
qrConvTri converged after 24 iterations
qrConv converged after 24 iterations
iter = 6
qrConvTri converged after 83 iterations
qrConv converged after 83 iterations
iter = 7
qrConvTri converged after 87 iterations
qrConv converged after 87 iterations
iter = 8
qrConvTri converged after 126 iterations
qrConv converged after 126 iterations
iter = 9
qrConvTri converged after 262 iterations
qrConv converged after 262 iterations
iter = 10
qrConvTri converged after 54 iterations
qrConv converged after 54 iterations
iter = 11
qrConvTri converged after 51 iterations
qrConv converged after 51 iterations
iter = 12
qrConvTri converged after 185 iterati

┌ Warning: Convergence hasnt been reached
└ @ Main In[21]:53


qrConvTri converged after 602 iterations
qrConv converged after 958 iterations
iter = 54
qrConvTri converged after 809 iterations
qrConv converged after 810 iterations
iter = 55
qrConvTri converged after 1105 iterations
qrConv converged after 1104 iterations
iter = 56
qrConvTri converged after 1059 iterations
qrConv converged after 1013 iterations
iter = 57
qrConvTri converged after 1929 iterations
qrConv converged after 1931 iterations
iter = 58
qrConvTri converged after 736 iterations
qrConv converged after 736 iterations
iter = 59
qrConvTri converged after 1011 iterations
qrConv converged after 1013 iterations
iter = 60
qrConvTri converged after 976 iterations
qrConv converged after 945 iterations
iter = 61
qrConvTri converged after 307 iterations
qrConv converged after 307 iterations
iter = 62
qrConvTri converged after 1218 iterations
qrConv converged after 1218 iterations
iter = 63
qrConvTri converged after 985 iterations
qrConv converged after 1117 iterations
iter = 64
qrConvTri 

┌ Warning: Convergence hasnt been reached
└ @ Main In[21]:53


In [29]:
draw = hcat(Tri,Hou)

plot(X,draw, label=["Givens" "HouseHolder"], xlabel= "taille de la matrice (n*n)", ylabel="temps d'exécution en s")
savefig("compQR.png")